In [52]:
import os
from pydub.utils import mediainfo
import random


In [104]:
def get_sample_rate(src):
    info = mediainfo(src)
    # print(info)
    return int(info["sample_rate"])


def get_total_samples(src):
    info = mediainfo(src)
    # print(info)
    assert info["channels"] == "2" and info["sample_rate"] == "44100"
    return int(info["duration_ts"])

def cut(src, dest, start_sample, end_sample):
    # print("cut", int(start_sample*(sample_rate / 22050)), int(start_sample*(sample_rate / 22050)) + (end_sample-start_sample))
    os.system(f'ffmpeg -i "{src}" -ac 1 -ar 22050 -af atrim=start_sample={start_sample * 2}:end_sample={end_sample * 2} "{dest}" -y')

In [117]:
CHUNK_SIZE = 661500
def process(src, alias, onlyone=False):
    total_samples = get_total_samples(src) // 2
    # print(total_samples, sample_rate)
    if (onlyone):
        i = random.randint(0, total_samples // CHUNK_SIZE - 1)
        cut(src, f"{alias}.wav", CHUNK_SIZE * i, CHUNK_SIZE * (i+1))
    else:
        for i in range(total_samples // CHUNK_SIZE): 
            cut(src, f"{alias}_{i}.wav", CHUNK_SIZE * i, CHUNK_SIZE * (i+1))

In [ ]:
for i, filename in  enumerate(filter(lambda x: ".m4a" in x, os.listdir("raw_speech"))):
    print(i, filename)
    process("raw_speech/"+filename, "xxm_speech/"+str(i))


In [ ]:
# Resample to 44.1K
file_list = list(filter(lambda x: ".m4a" in x, os.listdir("../xxm_collection/bili0901")))

for file in file_list:
    sample_rate = get_sample_rate("../xxm_collection/bili0901/"+file)
    print(file, sample_rate)
    if (sample_rate == 44100):
        os.system(f'cp "../xxm_collection/bili0901/{file}" raw_singing/')
    else:
        os.system(f'ffmpeg -i "../xxm_collection/bili0901/{file}" -ar 44100 raw_singing/{file}')
# for i, filename in enumerate(file_list):
#     print(i, filename)
#     process("../xxm_collection/bili0901/"+filename, "xxm_singing/"+str(i))

In [ ]:
file_list = list(filter(lambda x: ".m4a" in x, os.listdir("raw_singing")))
file_list = random.choices(file_list, k=137)
file_list

for i, filename in  enumerate(file_list):
    print(i, filename)
    process("raw_singing/"+filename, "xxm_singing/"+str(i), onlyone=True)